In [ ]:
import sys

sys.path.append("../")
from config import *

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!cd "{PROJECT_ROOT}"
sys.path.append("./src/")
!pwd

In [ ]:
!pip install -r "{PROJECT_ROOT}/requirements.txt"

In [ ]:
!dvc pull -f

In [ ]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader
from tqdm import trange 
from transformers import AutoConfig, AutoTokenizer
import torch.nn as nn
from torch import optim
import torch.nn as nn
from scipy import stats
import os
from pathlib import Path
from model_utils import BertRegresser, Excerpt_Dataset, map2index, map_model

In [ ]:
## Model Configurations

MODEL_OUT_DIR = '/content/drive/MyDrive/Thesis/output/sciBert_Kaggle/'

p = {
    'max_len': 512,
    'batch_size': 6,
    'lr' : 4.0638e-05,
    'epochs': 18,
    'dropout': 0.5,
    'num_threads' : 1,
    'model_name' : 'allenai/scibert_scivocab_uncased',
    #'model_name' : 'bert-base-uncased',
    'do_train' : False,
    'random_seed': 24
}

## Fine Tuning

In [ ]:
## Configuration loaded from AutoConfig 
aconfig = AutoConfig.from_pretrained(p['model_name'])
## Tokenizer loaded from AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(p['model_name'])
## Creating the model from the desired transformer model
model = BertRegresser(aconfig, dropout=p["dropout"])

In [ ]:
#freeze all layers except regression head

"""unfreeze_layers = ['bert.pooler', 'regressor.1']
for name, params in model.named_parameters():
  params.requires_grad = False
  for ele in unfreeze_layers:
    if ele in name:
      params.requires_grad = True
      break

for name, params in model.named_parameters():
  if params.requires_grad:
    print(name, params.size())"""

In [ ]:
for name, params in model.named_parameters():
    if params.requires_grad:
        print(name, params.size())

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
## Putting model to device
model = model.to(device)
## Takes as the input the logits of the positive class and computes the binary cross-entropy 
# criterion = nn.BCEWithLogitsLoss()
criterion = nn.MSELoss()
## Optimizer
optimizer = optim.Adam(params=model.parameters(), lr=p['lr'])

### Load data

In [ ]:
text_map = pd.read_csv(f'{DATA_DIR}/annotated/Kopie von 140_annotations_pairs.csv', index_col=0)
scores = pd.read_csv(f'{DATA_DIR}/annotated/Kopie von 140_humanannotation_withoutunannotated.csv', index_col=0)

In [ ]:
abstract_df =text_map[['abstract']]
title_df = text_map[['title', 'bart_base', 'bart_cnn', 'bart_xsum', 't5_small', 'gpt2', 'pegasus_xsum']]
abstract_np = abstract_df.to_numpy()
scores_np = scores.to_numpy()
title_np = title_df.to_numpy()
idx_map = [map2index(row) for row in map_model[:140]]
title_np_picked = np.array([np.take(row1, np.sort(row2)) for row1, row2 in zip(title_np, idx_map)])
score_np_picked = np.array([np.take(row1, np.sort(row2)) for row1, row2 in zip(scores_np, idx_map)])

In [ ]:
pairs_np_picked = np.concatenate([abstract_np, title_np_picked,score_np_picked], axis=1)
pairs_np_picked_shuffled = np.random.permutation(pairs_np_picked)

In [ ]:
abstracs = pairs_np_picked_shuffled[:,:1]
titles = pairs_np_picked_shuffled[:,1:7]
scores = pairs_np_picked_shuffled[:,7:].astype(float)
scores = np.around(scores, 4)

In [ ]:
lst = []

for ab, row1, row2 in zip(abstracs, titles, scores):
    assert len(row1) == len(row2)
    assert len(row1) == 6
    for t, s in zip(row1, row2):

        if np.isnan(s):
            print('found nan score')
        if t=='':
            print('found empty title')
        lst.append([ab[0] + '[SEP]' + t, s])
df = pd.DataFrame(np.array(lst))

In [ ]:
df.columns = ['excerpt', 'target']
#dataframe = dataframe.sample(frac=1, random_state=42).reset_index(drop=True)

In [ ]:
dftrain = df[:600].reset_index(drop=True)
dfdev = df[600:660].reset_index(drop=True)
dftest = df[660:].reset_index(drop=True)

Path(f'{OUTPUT_DIR}/reward_model_robust_test/').mkdir(parents=True, exist_ok=True)

dftrain.to_csv(f'{OUTPUT_DIR}/reward_model_robust_test/sciBert_shuffled_train.csv')
dfdev.to_csv(f'{OUTPUT_DIR}/reward_model_robust_test/sciBert_shuffled_dev.csv')
dftest.to_csv(f'{OUTPUT_DIR}/reward_model_robust_test/sciBert_shuffled_test.csv')

### Training Dataset

In [ ]:
train_set = Excerpt_Dataset(data=dftrain, maxlen=p['max_len'], tokenizer=tokenizer)
dev_set = Excerpt_Dataset(data=dfdev, maxlen=p['max_len'], tokenizer=tokenizer)
test_set = Excerpt_Dataset(data=dftest, maxlen=p['max_len'], tokenizer=tokenizer)

### Data Loaders

In [ ]:
train_loader = DataLoader(dataset=train_set, batch_size=p['batch_size'], num_workers=p['num_threads'], shuffle=True)
dev_loader = DataLoader(dataset=dev_set, batch_size=p['batch_size'], num_workers=p['num_threads'], shuffle=True)
test_loader = DataLoader(dataset=test_set, batch_size=p['batch_size'], num_workers=p['num_threads'], shuffle=True)

In [ ]:
# Do Train (do not use this for training of reward model, reward model trained using ray tune)

if p['do_train']:
    model.train(
        criterion=criterion,
        optimizer=optimizer,
        train_loader=train_loader,
        val_loader=dev_loader,
        epochs = p['epochs'],
        device = device
    )


### Load best read model

In [ ]:
model_state, optimizer_state = torch.load(os.path.join(f'{PROJECT_ROOT}/reward model', "checkpoint"))
model.load_state_dict(model_state)

### Title prediction function

In [ ]:
def predict(model, dataloader, device):
    predicted_label = []
    actual_label = []
    with torch.no_grad():
        for input_ids, attention_mask, target in (dataloader):
            
            input_ids, attention_mask, target = input_ids.to(device), attention_mask.to(device), target.to(device)
            output = model(input_ids, attention_mask)
                        
            predicted_label += output
            actual_label += target
            
    return predicted_label, actual_label


## Display Correlation

In [ ]:
output,GS_label = predict(model, train_loader, device)
cpu_output = np.array([x.cpu().data.numpy() for x in output]).squeeze()
cpu_target = np.array([x.cpu().data.numpy() for x in GS_label]).squeeze()
stats.spearmanr(cpu_output, cpu_target)[0]

In [ ]:
dev_output,dev_GS_label = predict(model, dev_loader, device)
cpu_dev_output = np.array([x.cpu().data.numpy() for x in dev_output]).squeeze()
cpu_dev_target = np.array([x.cpu().data.numpy() for x in dev_GS_label]).squeeze()
stats.spearmanr(cpu_dev_output, cpu_dev_target)[0]

In [ ]:
test_output,test_GS_label = predict(model, test_loader, device)
cpu_test_output = np.array([x.cpu().data.numpy() for x in test_output]).squeeze()
cpu_test_target = np.array([x.cpu().data.numpy() for x in test_GS_label]).squeeze()
stats.spearmanr(cpu_test_output, cpu_test_target)[0]

In [ ]:
dev_output,dev_GS_label = predict(model, dev_loader, device)
cpu_dev_output = np.array([x.cpu().data.numpy() for x in dev_output]).squeeze()
cpu_dev_target = np.array([x.cpu().data.numpy() for x in dev_GS_label]).squeeze()
stats.spearmanr(cpu_dev_output, cpu_dev_target)[0]

In [ ]:
test_output,test_GS_label = predict(model, test_loader, device)
cpu_test_output = np.array([x.cpu().data.numpy() for x in test_output]).squeeze()
cpu_test_target = np.array([x.cpu().data.numpy() for x in test_GS_label]).squeeze()
stats.spearmanr(cpu_test_output, cpu_test_target)[0]